In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import os

In [ ]:
images=np.load(open('/content/drive/MyDrive/All fruits/fruit_new.npy','rb'))
target=np.load(open('/content/drive/MyDrive/All fruits/target_new.npy','rb'))
images.shape,target.shape

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(images,target,random_state=5,stratify=target)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

Model

In [ ]:
from keras.applications.vgg19 import VGG19
vgg_model=VGG19(weights='imagenet')
vgg_model.summary()


574726144/574710816 [==============================] - 4s 0us/step
Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

In [ ]:
for layer in vgg_model.layers:
  layer.trainable=False

In [ ]:
from keras.layers import Flatten, Dense, Dropout
from keras.models import Model
flat=Flatten(name='flat')(vgg_model.layers[-2].output)
h1=Dense(1024,activation='relu')(flat)
h2=Dense(128,activation='relu')(h1)
output=Dense(50,activation='softmax')(h2)
model=Model(inputs=vgg_model.inputs,outputs=output)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [ ]:
from keras import optimizers

In [ ]:
model.compile(optimizer=optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=1)
checkpoint = ModelCheckpoint('/content/drive/MyDrive/All fruits/temp2/FruitsNewBestModel1.h5', monitor='val_accuracy', verbose=1, save_best_only=True,
                                 mode='auto')

In [ ]:
history=model.fit(X_train,y_train,epochs=40,validation_data=(X_test,y_test),callbacks=[checkpoint,early_stopper])

Epoch 1/40
784/784 [==============================] - 222s 222ms/step - loss: 1.8203 - accuracy: 0.5565 - val_loss: 0.2007 - val_accuracy: 0.9467

Epoch 00001: val_accuracy improved from -inf to 0.94665, saving model to /content/drive/MyDrive/All fruits/temp2/FruitsNewBestModel1.h5
Epoch 2/40
784/784 [==============================] - 166s 212ms/step - loss: 0.1571 - accuracy: 0.9585 - val_loss: 0.2087 - val_accuracy: 0.9389

Epoch 00002: val_accuracy did not improve from 0.94665
Epoch 3/40
784/784 [==============================] - 165s 211ms/step - loss: 0.1189 - accuracy: 0.9661 - val_loss: 0.1469 - val_accuracy: 0.9604

Epoch 00003: val_accuracy improved from 0.94665 to 0.96041, saving model to /content/drive/MyDrive/All fruits/temp2/FruitsNewBestModel1.h5
Epoch 4/40
784/784 [==============================] - 165s 211ms/step - loss: 0.1192 - accuracy: 0.9656 - val_loss: 0.1464 - val_accuracy: 0.9639

Epoch 00004: val_accuracy improved from 0.96041 to 0.96388, saving model to /conte

Model Evaluation

In [ ]:
import numpy as np
images=np.load(open('/content/drive/MyDrive/All fruits/fruit_new.npy','rb'))
target=np.load(open('/content/drive/MyDrive/All fruits/target_new.npy','rb'))
images.shape,target.shape

((33440, 224, 224, 3), (33440, 50))

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(images,target,random_state=5,stratify=target)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((25080, 224, 224, 3), (8360, 224, 224, 3), (25080, 50), (8360, 50))

In [ ]:
del images 
del target

In [ ]:
import keras
model=keras.models.load_model('/content/drive/MyDrive/All fruits/temp2/FruitsVggNewModel.h5')

In [ ]:
y_train_pred=model.predict(X_train)
y_test_pred=model.predict(X_test)

In [ ]:
y_train_pred=np.argmax(y_train_pred,axis=-1)
y_train_label=np.argmax(y_train,axis=-1)

y_test_pred=np.argmax(y_test_pred,axis=-1)
y_test_label=np.argmax(y_test,axis=-1)
y_test_pred,y_test_label

(array([42, 42, 42, ..., 42, 42, 42]), array([49, 30, 49, ...,  9,  2, 23]))

In [ ]:
from sklearn.metrics import accuracy_score,classification_report
train_acc=accuracy_score(y_train_pred,y_train_label)
test_acc=accuracy_score(y_test_pred,y_test_label)
train_acc,test_acc

(0.024362041467304626, 0.024401913875598084)

In [ ]:
report=classification_report(y_test_label,y_test_pred)
print(report)

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       198
           1       0.93      0.93      0.93       150
           2       1.00      1.00      1.00       200
           3       0.99      0.89      0.94       175
           4       0.99      1.00      0.99       200
           5       0.99      0.99      0.99       200
           6       1.00      1.00      1.00       200
           7       1.00      0.96      0.98       150
           8       0.97      0.93      0.95       200
           9       0.93      0.94      0.94       150
          10       0.97      0.88      0.92       150
          11       0.94      0.99      0.97       150
          12       1.00      0.97      0.99       200
          13       0.99      0.96      0.97       150
          14       0.98      1.00      0.99       125
          15       0.98      1.00      0.99       125
          16       0.98      1.00      0.99       131
          17       1.00    

Model Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

cm=confusion_matrix(y_test_label,y_test_pred)
print(cm)

[[198   0   0 ...   0   0   0]
 [  0 139   0 ...   0   0   0]
 [  0   0 200 ...   0   0   0]
 ...
 [  0   0   0 ... 200   0   0]
 [  0   0   0 ...   0 190   0]
 [  0   0   0 ...   0   0 200]]


In [ ]:
import os
fruit_lt=os.listdir('/content/drive/MyDrive/All fruits/Fruit')
fruit_lt,len(fruit_lt)

(['Acerola_fruit',
  'Apple_fruit',
  'Apricots_fruit',
  'Banana_fruit',
  'Blackberry_fruit',
  'Blueberry_fruit',
  'Carambola_fruit',
  'Cherry_fruit',
  'Chestnut_fruit',
  'Coconut_fruit',
  'Custard_apple_fruit',
  'Date_fruit',
  'Dragon_Fruit',
  'Fig_fruit',
  'Gooseberry_fruit',
  'Grapes_fruit',
  'Guava_fruit',
  'Jack_fruit',
  'Jambolan_fruit',
  'Jujube_fruit',
  'Kiwi_fruit',
  'Kumquat_fruit',
  'Lime_fruit',
  'Litchi_fruit',
  'Loquat_fruit',
  'Mamey_fruit',
  'Mango_fruit',
  'Mulberry_fruit',
  'Muskmelon_fruit',
  'Nectrine_fruit',
  'Olive_fruit',
  'Orange_fruit',
  'Papaya_fruit',
  'Pear_fruit',
  'Pineapple_fruit',
  'Plum_fruit',
  'Pomegranate_fruit',
  'Pummelo_fruit',
  'Peach_fruit',
  'Quince_fruit',
  'Rambutan_Fruit',
  'Sapodilla_fruit',
  'Soursop_fruit',
  'Strawberry_fruit',
  'Ugli_Fruit',
  'Ugni_fruit',
  'Water_Chestnut_fruit',
  'Watermelon_fruit',
  'White_Mulberry_fruit',
  'Yangmei_fruit'],
 50)

In [ ]:
fruit_map={}
for index,val in enumerate(fruit_lt):
  fruit_map[index]=val
fruit_map

{0: 'Acerola_fruit',
 1: 'Apple_fruit',
 2: 'Apricots_fruit',
 3: 'Banana_fruit',
 4: 'Blackberry_fruit',
 5: 'Blueberry_fruit',
 6: 'Carambola_fruit',
 7: 'Cherry_fruit',
 8: 'Chestnut_fruit',
 9: 'Coconut_fruit',
 10: 'Custard_apple_fruit',
 11: 'Date_fruit',
 12: 'Dragon_Fruit',
 13: 'Fig_fruit',
 14: 'Gooseberry_fruit',
 15: 'Grapes_fruit',
 16: 'Guava_fruit',
 17: 'Jack_fruit',
 18: 'Jambolan_fruit',
 19: 'Jujube_fruit',
 20: 'Kiwi_fruit',
 21: 'Kumquat_fruit',
 22: 'Lime_fruit',
 23: 'Litchi_fruit',
 24: 'Loquat_fruit',
 25: 'Mamey_fruit',
 26: 'Mango_fruit',
 27: 'Mulberry_fruit',
 28: 'Muskmelon_fruit',
 29: 'Nectrine_fruit',
 30: 'Olive_fruit',
 31: 'Orange_fruit',
 32: 'Papaya_fruit',
 33: 'Pear_fruit',
 34: 'Pineapple_fruit',
 35: 'Plum_fruit',
 36: 'Pomegranate_fruit',
 37: 'Pummelo_fruit',
 38: 'Peach_fruit',
 39: 'Quince_fruit',
 40: 'Rambutan_Fruit',
 41: 'Sapodilla_fruit',
 42: 'Soursop_fruit',
 43: 'Strawberry_fruit',
 44: 'Ugli_Fruit',
 45: 'Ugni_fruit',
 46: 'Water_C

In [ ]:
for index,row in enumerate(cm):
  count=0
  for j in range(len(row)):
    if row[j]>0:
      count+=1
  print(count)
  break


1


In [ ]:
for index,row in enumerate(cm):
  count=0
  for j in range(len(row)):
    if(row[j]>0):
      count+=1
  if count>1:
    print(f'{fruit_map[index]}--->{row}')
  

Apple_fruit--->[  0 139   0   1   0   0   0   0   1   0   1   1   0   1   1   0   1   0
   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   1   0   0
   0   1   0   1   0   0   0   0   0   0   0   0   0   0]
Banana_fruit--->[  0   5   0 155   0   1   0   0   0   2   0   1   0   0   0   1   1   0
   0   0   0   0   0   0   2   0   1   0   0   0   1   0   0   0   0   2
   0   0   0   2   0   0   0   0   0   0   0   0   0   1]
Blueberry_fruit--->[  0   0   0   0   0 198   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   2   0   0   0   0   0]
Cherry_fruit--->[  1   1   0   0   0   0   0 144   0   0   0   1   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   2   0   0   0   0   0   0   0   0
   0   1   0   0   0   0   0   0   0   0   0   0   0   0]
Chestnut_fruit--->[  0   0   0   0   0   0   0   0 185   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   